In [190]:
%matplotlib inline
import glob
import pandas as pd
import textwrap

In [191]:
d = pd.read_excel('cius_data/cius2012datatables/Table_77_Full_time_Law_Enforcement_Employess_by_State_2012.xls')

In [192]:
FILE_MAPPING = {
    '2000': 'cius_data/2000/table2-5_vicage00.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2000
    '2001': 'cius_data/2001/table2-5_vicage01.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2001
    '2002': 'cius_data/2002/table2-5_vicage02.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2002
    '2003': 'cius_data/2003/table2-4_age03.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2003/
    '2004': 'cius_data/2004/04tbl77a.xls', # https://www2.fbi.gov/ucr/cius_04/law_enforcement_personnel/table_77.html
    '2005': 'cius_data/2005/05tbl77.xls',
    '2006': 'cius_data/CIUS2006datatables/06tbl77.xls',
    '2007': 'cius_data/CIUS2007datatables/07tbl77.xls',
    '2008': 'cius_data/CIUS2008datatables/08tbl77.xls',
    '2009': 'cius_data/CIUS2009datatables/09tbl77.xls',
    '2010': 'cius_data/CIUS2010datatables/10tbl77.xls',
    '2011': 'cius_data/CIUS2011datatables/Table_77_Full-time_Law_Enforcement_Employees_by_State_2011.xls',
    '2012': 'cius_data/cius2012datatables/Table_77_Full_time_Law_Enforcement_Employess_by_State_2012.xls',
    '2013': 'cius_data/cius2013datatables/Table_77_Full_time_Law_Enforcement_Employess_by_State_2013.xls'
}

YEARS = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

def count_by_year(year):
    file_path = FILE_MAPPING[year]
    raw = pd.read_excel(file_path)
    raw = raw[['Table 77', 'Unnamed: 7', 'Unnamed: 6', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']]
    raw.columns = ['State', 'Population', 'Number of agencies', 'Total law enforcement employees', 'Total male officers', 'Total female officers', 'Total male civilians', 'Total female civilians']
    raw['Year'] = year
    data = raw.iloc[4:, :]
    data.set_index('Year', inplace=True)
    return data[data['State'].notnull()]

years = pd.concat([count_by_year(y) for y in YEARS])
years['State'] = years['State'].apply(lambda x: x.strip())

In [193]:
def file_for_year(year):
    file = FILE_MAPPING[year].split('/')[-1]
    return '%s: %s' % (year, file)

files = [file_for_year(y) for y in YEARS]

def write_notes(filepath, body, footnotes):
    txt = textwrap.dedent("""\
# %s from %s to %s

Generated from the following tables of the corresponding year's Crime in the US data table download:
%s

Footnotes/caveats:
%s
""" % (body, YEARS[0], YEARS[-1], '\n'.join(files), '\n'.join(footnotes)))
    with open(filepath, 'w') as f:
        f.write(txt)

In [194]:
dest_path = 'cius_data/processed/table-77-fte-law-enforcement/%s.csv'
txt_path = dest_path.replace('.csv', '.txt')
years_file = 'police-employee-counts-by-state-%s-%s' % (YEARS[0], YEARS[-1])
years.to_csv(dest_path % years_file)

national_cols = ['Population', 'Total law enforcement employees', 'Total male officers', 'Total female officers', 'Total male civilians', 'Total female civilians']
national = years.reset_index()
national = national.groupby('Year')[national_cols].sum()
national_file = 'police-employee-counts-nationally-%s-%s' % (YEARS[0], YEARS[-1])
national.to_csv(dest_path % national_file)

In [195]:
years_body = 'Yearly totals for law enforcement employee counts by state'
years_footnotes = ['Some states are missing some years. For example, West Virginia did not submit 2008 totals']
write_notes(txt_path % years_file, years_body, years_footnotes)

national_body = 'Yearly totals for law enforcement employee counts nationally'
national_footnotes = ['Population count generated by summing reported state population totals, not from national population estimates']
write_notes(txt_path % national_file, national_body, national_footnotes)